In [1]:
import pandas as pd
import MetaTrader5 as mt5
from datetime import datetime

from secret_login import Secret_Login, IC_Markets_Login

In [3]:
mt5.initialize()

True

## Login


In [8]:
print(mt5.account_info().login)
print(mt5.account_info().server)

51750680
ICMarketsSC-Demo


In [9]:
temp = IC_Markets_Login()

account = mt5.account_info().login

password = temp.password

server = mt5.account_info().server

print(f"Account : {account}")
print(f"password : {password}")
print(f"Server : {server}")



Account : 51750680
password : o&bCOUrc8jTHs8
Server : ICMarketsSC-Demo


In [16]:
authorized = mt5.login(account,password,server)

if authorized:
    print("connected to account #{}".format(account))
else:
    print("failed to connect at account #{}, error code: {}".format(account, mt5.last_error()))

connected to account #51750680


# __Retrieving Data__: The API way

In [25]:
ticker = 'EURUSD'
frequency = mt5.TIMEFRAME_M1
from_date = datetime.now()
number_of_rows = 10000


In [24]:
rates = mt5.copy_rates_from(ticker, frequency, from_date,number_of_rows )

rates

In [19]:
range_rates = mt5.copy_rates_range(ticker,frequency,datetime(2024,1,5), from_date)
range_rates

In [20]:
data = pd.DataFrame(rates)
data

,time,open,high,low,close,tick_volume,spread,real_volume
0,1712141820,1.07703,1.07709,1.07699,1.07707,47,0,0
1,1712141880,1.07708,1.07708,1.07696,1.07703,40,0,0
2,1712141940,1.07703,1.07707,1.07701,1.07707,28,0,0
3,1712142000,1.07707,1.07714,1.07702,1.07710,49,0,0
4,1712142060,1.07710,1.07720,1.07707,1.07720,70,0,0
...,...,...,...,...,...,...,...,...
9995,1712915100,1.06851,1.06865,1.06842,1.06844,110,0,0
9996,1712915160,1.06844,1.06845,1.06829,1.06845,109,0,0
9997,1712915220,1.06845,1.06865,1.06842,1.06861,89,0,0
9998,1712915280,1.06861,1.06862,1.06838,1.06847,81,0,0


In [21]:
data['time'] = pd.to_datetime(data['time'],unit = 's')
data

,time,open,high,low,close,tick_volume,spread,real_volume
0,2024-04-03 10:57:00,1.07703,1.07709,1.07699,1.07707,47,0,0
1,2024-04-03 10:58:00,1.07708,1.07708,1.07696,1.07703,40,0,0
2,2024-04-03 10:59:00,1.07703,1.07707,1.07701,1.07707,28,0,0
3,2024-04-03 11:00:00,1.07707,1.07714,1.07702,1.07710,49,0,0
4,2024-04-03 11:01:00,1.07710,1.07720,1.07707,1.07720,70,0,0
...,...,...,...,...,...,...,...,...
9995,2024-04-12 09:45:00,1.06851,1.06865,1.06842,1.06844,110,0,0
9996,2024-04-12 09:46:00,1.06844,1.06845,1.06829,1.06845,109,0,0
9997,2024-04-12 09:47:00,1.06845,1.06865,1.06842,1.06861,89,0,0
9998,2024-04-12 09:48:00,1.06861,1.06862,1.06838,1.06847,81,0,0


In [22]:
print(f'first time : \n{data.head(1)['time']}')
print(f"\nLast time : \n{data.tail(1)['time'] }")

first time : 
0   2024-04-03 10:57:00
Name: time, dtype: datetime64[ns]

Last time : 
9999   2024-04-12 09:49:00
Name: time, dtype: datetime64[ns]


# __Retrieving Data__: The Manual way


In [4]:
manual_data = pd.read_csv('EURUSD_M1_2004_2024.csv', delimiter= '\t')

In [5]:
manual_data.shape

(7413614, 9)

In [6]:
manual_data.head()

,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<TICKVOL>,<VOL>,<SPREAD>
0,2004.01.01,00:04:00,1.2592,1.2592,1.2592,1.2592,1,0,40
1,2004.01.01,00:06:00,1.2596,1.2596,1.2596,1.2596,1,0,40
2,2004.01.01,00:14:00,1.2597,1.2597,1.2597,1.2597,1,0,40
3,2004.01.01,01:02:00,1.2595,1.2595,1.2595,1.2595,1,0,40
4,2004.01.01,01:06:00,1.2596,1.2596,1.2596,1.2596,1,0,40
